# Метод опорных векторов (SVM): математика и реализация с нуля

В этом ноутбуке мы разберём **метод опорных векторов (Support Vector Machine, SVM)** — один из самых фундаментальных алгоритмов машинного обучения.

- **Цель**: найти **оптимальную разделяющую гиперплоскость**.
- **Ключевая идея**: **максимизировать зазор (margin)** между классами.
- **Опорные вектора**: точки, ближайшие к границе — они **определяют положение гиперплоскости**.

Реализуем **soft-margin SVM** с градиентным спуском на чистом NumPy.

## 1. Формулировка задачи

Рассматриваем **бинарную классификацию**:
- Объект: $\mathbf{x} \in \mathbb{R}^d$
- Метка: $y \in \{-1, +1\}$

Модель предсказывает:
$$
F(\mathbf{x}) = \text{sign}(\mathbf{w}^\top \mathbf{x} - b)
$$

Наша задача — найти такие $\mathbf{w}$ и $b$, чтобы:
- Все объекты были **правильно классифицированы**,
- **Зазор между классами был максимальным**.

## 2. Зазор (Margin) и опорные вектора

Для объекта $(\mathbf{x}_i, y_i)$ **отступ (margin)** определяется как:
$$
M_i = y_i (\mathbf{w}^\top \mathbf{x}_i - b)
$$

Интерпретация:
- $M_i > 1$: объект **правильно классифицирован** и **вне разделяющей полосы**,
- $0 < M_i < 1$: объект **внутри полосы**, но с правильным знаком,
- $M_i \leq 0$: объект **ошибочно классифицирован**.

**Опорные вектора** — это объекты, для которых $M_i < 1$.

## 3. Жёсткий зазор (Hard-Margin SVM)

Если данные **линейно разделимы**, решается задача:
$$
\begin{cases}
\dfrac{1}{2} \|\mathbf{w}\|^2 \to \min \\
y_i (\mathbf{w}^\top \mathbf{x}_i - b) \geq 1, \quad \forall i
\end{cases}
$$

Это задача **квадратичного программирования (QP)**.  
Решается через **двойственную функцию Лагранжа** → **метод SMO** (используется в `sklearn`).

**Недостаток**: не работает при **шуме** или **перекрытии классов**.

## 4. Мягкий зазор (Soft-Margin SVM)

Для **реальных данных** вводим **слабые ограничения** через **переменные зазора** $\xi_i \geq 0$:

$$
\begin{cases}
\dfrac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^m \xi_i \to \min \\
y_i (\mathbf{w}^\top \mathbf{x}_i - b) \geq 1 - \xi_i, \quad \forall i
\end{cases}
$$

- $C > 0$ — гиперпараметр: **баланс между максимальным зазором и допустимыми ошибками**.
- Чем больше $C$, тем **меньше ошибок допускается** (ближе к hard-margin).

Эту задачу можно **аппроксимировать** через **функцию потерь**.

## 5. Функция потерь: Hinge Loss

Мягкий зазор эквивалентен минимизации **безусловной функции**:
$$
Q(\mathbf{w}) = \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^m \max(0, 1 - y_i (\mathbf{w}^\top \mathbf{x}_i - b))
$$

- $\max(0, 1 - M_i)$ — **Hinge Loss**.
- Если $M_i \geq 1$ → $H = 0$ → **градиенты не текут**.
- Если $M_i < 1$ → $H > 0$ → **веса обновляются**.

Это позволяет использовать **градиентный спуск**!

## 6. Градиенты для обновления весов

Градиент функции потерь по $\mathbf{w}$:
$$
\nabla Q =
\begin{cases}
\mathbf{w} - C \cdot y_i \mathbf{x}_i, & \text{если } y_i (\mathbf{w}^\top \mathbf{x}_i - b) < 1 \\
\mathbf{w}, & \text{иначе}
\end{cases}
$$

В нашей реализации:
- $\alpha = 1/C$ (коэффициент регуляризации),
- Обновление: $\mathbf{w} \leftarrow \mathbf{w} - \eta \cdot \nabla Q$,
- где $\eta$ — **скорость обучения (learning rate)**.

## 7. Почему Hinge Loss максимизирует зазор?

- Градиенты текут **только для объектов с $M_i < 1$** — то есть для **опорных векторов**.
- Объекты с $M_i \geq 1$ **не влияют на обучение** → веса не меняются.
- Таким образом, алгоритм «заботится» **только о ближайших к границе точках** — это и есть суть SVM.

## 8. Отличия от логистической регрессии

| Характеристика        | SVM                          | Логистическая регрессия      |
|----------------------|------------------------------|-------------------------------|
| Функция потерь       | Hinge Loss: $\max(0, 1 - M)$ | Log Loss: $\log(1 + e^{-M})$ |
| Выход                | Метка класса ($\pm1$)        | Вероятность                   |
| Интерпретация        | Максимизация зазора          | Максимизация правдоподобия   |
| Опорные объекты      | Только опорные вектора       | Все объекты влияют            |

## 9. Заключение

- Мы реализовали **soft-margin SVM** через **градиентный спуск**.
- Код не решает QP-задачу, но **аппроксимирует её** через Hinge Loss.
- Это **практичный и обучающий подход**, позволяющий понять **суть SVM "под капотом"**.

Полный код реализации — в `src/manually_svm.py`.  
Демонстрация на данных Ирисов — в `examples/demo_svm.py`.